In [ ]:
import os
import numpy as np
import mne
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dropout, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from multiprocessing import Pool
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold


# Constants
NUM_CHANNELS = 64
SAMPLING_RATE = 250
TRANSIENT_REMOVAL_SAMPLES = 1000
RECORDING_DURATION_SEC = 300
SEGMENT_DURATION_SEC = 10  # 10-second segments
N_SAMPLES_PER_SEGMENT = SEGMENT_DURATION_SEC * SAMPLING_RATE
FIXED_SEGMENTS_STARTS = [
    i * N_SAMPLES_PER_SEGMENT for i in range(RECORDING_DURATION_SEC // SEGMENT_DURATION_SEC)
]

# Paths
DATA_DIRS = {"HC": "/content/drive/MyDrive/UBCData/HCraw", "AC": "/content/drive/MyDrive/UBCData/ACraw"}
TEMP_DIR = "./content/drive/MyDrive/UBCData/unzipped_files"

# File extraction
def unzip_file(file_path, output_dir):
    if file_path.endswith(".zip"):
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        extracted_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".mff")]
        return extracted_files[0] if extracted_files else None
    return file_path

# Data processing
def process_file(file_info):
    file_path, label = file_info
    try:
        raw = mne.io.read_raw_egi(file_path, preload=True, events_as_annotations=True)

        # Apply a low-pass filter
        raw.filter(None, 120, fir_design='firwin', skip_by_annotation='edge')

        raw_data = raw.get_data()

        if raw_data.shape[1] <= 2 * TRANSIENT_REMOVAL_SAMPLES:
            return [], []

        cleaned_data = raw_data[:, TRANSIENT_REMOVAL_SAMPLES:-TRANSIENT_REMOVAL_SAMPLES]
        total_samples = RECORDING_DURATION_SEC * SAMPLING_RATE

        if cleaned_data.shape[1] < total_samples:
            return [], []

        # Fixed segments only
        fixed_segments = [
            cleaned_data[:, start:start + N_SAMPLES_PER_SEGMENT]
            for start in FIXED_SEGMENTS_STARTS
            if start + N_SAMPLES_PER_SEGMENT <= cleaned_data.shape[1]
        ]

        for i in range(len(fixed_segments)):
            if fixed_segments[i].shape[0] == 65:
                fixed_segments[i] = fixed_segments[i][:64, :]

        labels = [label] * len(fixed_segments)
        return fixed_segments, labels

    except Exception as e:
        print(f"Error processing {os.path.basename(file_path)}: {e}")
        return [], []

# Model training
def cross_validate_model(X_augmented, y_augmented, k=5, epochs=30, batch_size=32):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []

    for fold, (train_idx, test_idx) in enumerate(kfold.split(X_augmented)):
        print(f"Starting fold {fold + 1}/{k}...")

        X_train, X_test = X_augmented[train_idx], X_augmented[test_idx]
        y_train, y_test = y_augmented[train_idx], y_augmented[test_idx]

        X_train = X_train.reshape(X_train.shape[0], X_train.shape[2], X_train.shape[1])
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[2], X_test.shape[1])

        model = Sequential([
            Bidirectional(LSTM(100, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
            BatchNormalization(),
            Dropout(0.3),
            Bidirectional(LSTM(100)),
            BatchNormalization(),
            Dropout(0.3),
            Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
            Dense(1, activation='sigmoid', kernel_regularizer=l2(0.01)),
        ])

        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        print(f"Fold {fold + 1} accuracy: {accuracy * 100:.2f}%")
        fold_accuracies.append(accuracy)

    print(f"Average accuracy across {k} folds: {np.mean(fold_accuracies) * 100:.2f}%")
    return np.mean(fold_accuracies)

    # Plot training and validation accuracy
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='x')
    plt.title('Model Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid()
    plt.show()

    return model

# Main function
def main():
    X_augmented = []
    y_augmented = []

    os.makedirs(TEMP_DIR, exist_ok=True)
    file_info_list = []

    for label, folder in enumerate(DATA_DIRS.values()):
        for idx, file_name in enumerate(os.listdir(folder)):
            if idx >= 20:  # Limit files for faster processing
                break
            file_path = os.path.join(folder, file_name)
            file_info_list.append((file_path, label))

    with Pool(processes=4) as pool:  # Use multiprocessing for faster processing
        results = pool.map(process_file, file_info_list)

    for segments, labels in results:
        X_augmented.extend(segments)
        y_augmented.extend(labels)

    X_augmented = np.array(X_augmented, dtype=np.float32)
    y_augmented = np.array(y_augmented, dtype=np.float32)

    if len(X_augmented) > 0:
        cross_validate_model(X_augmented, y_augmented, epochs=40, batch_size=32)
        print(f"Augmentation complete with {len(X_augmented)} segments.")
    else:
        print("Augmentation complete. No valid segments were created.")

if __name__ == "__main__":
    main()


Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_0016.mff.zip...
Error processing HC_0016.mff.zip: 'ascii' codec can't decode byte 0x90 in position 0: ordinal not in range(128)
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_009.mff.zip...
Error processing HC_009.mff.zip: 'ascii' codec can't decode byte 0x90 in position 0: ordinal not in range(128)
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_001.raw...
    Reading events ...
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_004.raw...
    Reading events ...
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_0010.mff.zip...
Error processing HC_0010.mff.zip: 'ascii' codec can't decode byte 0x90 in position 0: ordinal not in range(128)
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_006.raw...
Reading EGI header from /content/drive/MyDrive/UBCData/HCraw/HC_003.raw...
    Reading events ...
    Reading events ...
    Assembling measurement info ..

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


Reading 0 ... 91421  =      0.000 ...   365.684 secs...
Reading 0 ... 91471  =      0.000 ...   365.884 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.2e+02 Hz

FIR filter parameters
---------------------
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.2e+02 Hz
Designing a one-pass, zero-phase, non-causal lowpass filter:

FIR filter parameters
- Windowed time-domain design (firwin) method
---------------------
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Upper passband edge: 120.00 Hz
- Windowed time-domain design (firwin) method
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 122.50 Hz)
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 120.00 Hz
- Filter length: 165 samples (0.660 s)
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 122.50 Hz)

- Filter l

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.4s


- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 122.50 Hz)
- Filter length: 165 samples (0.660 s)



...

Reading EGI header from /content/drive/MyDrive/UBCData/ACraw/AC_0018.RAW...
    Reading events ...
    Assembling measurement info ...
    Excluding events {} ...
Reading 0 ... 87388  =      0.000 ...   349.552 secs...
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 5.00 Hz (-6 dB cutoff frequency: 122.50 Hz)
- Filter length: 165 samples (0.660 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Starting fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Fold 1 accuracy: 100.00%
Starting fold 2/5...
Fold 2 accuracy: 68.52%
Starting fold 3/5...
Fold 3 accuracy: 100.00%
Starting fold 4/5...
Fold 4 accuracy: 100.00%
Starting fold 5/5...
Fold 5 accuracy: 96.30%
Average accuracy across 5 folds: 92.96%
Augmentation complete with 1080 segments.


In [ ]:
!pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.3 MB/s eta 0:00:00
